## Data loading

In [1]:
from mesh4d import obj3d

mesh_ls, _ = obj3d.load_mesh_series(
    folder='/Users/knpob/Territory/Kolmo/data/DynaFootLite/Fast',
    start=0,
    stride=1,
    end=4,
    load_texture=False,
)

[====================] 100.0% loading: /Users/knpob/Territory/Kolmo/data/DynaFootLite/Fast/Sub05_Fast_L1_102_143.000031.obj


In [2]:
from mesh4d import utils
from mesh4d.analyse.crave import clip_with_contour

landmarks = utils.load_pkl_object('output/fast.pkl')
landmarks.interp_field()

contour = landmarks.extract(('P7', 'P10', 'P11', 'P12'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=40, contour=contour, clip_bound='', margin=0, invert=True)

In [3]:
foot_ls = obj3d.init_obj_series(
    mesh_clip_ls,
    obj_type=obj3d.Obj3d_Deform
    )

## PCA axes

In [4]:
import numpy as np
from sklearn.decomposition import PCA

def pca_axes(vertices):
    pca = PCA()
    pca.fit(vertices)

    stds = np.sqrt(pca.explained_variance_)
    axes = pca.components_  # (n_axes, n_coords)
    mean = pca.mean_ # (n_coords)
    return mean, axes, stds

In [5]:
import pyvista as pv

def draw_axes(mean, axes, stds, mesh, is_export: bool = False, export_path: str = ''):
    scene = pv.Plotter()

    for i in range(3):
        start_point = mean
        end_point = mean + 3 * stds[i] * axes[i]
        scene.add_lines(np.array([start_point, end_point]), color='goldenrod')
        scene.add_point_labels(
            end_point, [f"PC{i + 1}@({axes[i][0]:.2f}, {axes[i][1]:.2f}, {axes[i][2]:.2f})var{stds[i]:.2f}"], 
            font_size=15, point_size=0, shape='rounded_rect',
            point_color='goldenrod', always_visible=True,
            )

    scene.add_mesh(mesh, opacity=0.5)
    scene.camera_position = 'zy'
    
    if is_export:
        scene.screenshot(export_path)
    else:
        scene.show()

In [6]:
import os

axes_ls = []

for idx in range(5):
    vertices = landmarks.to_array()[0][idx][:10]
    mean, axes, stds = pca_axes(vertices)
    axes_ls.append({'mean': mean, 'axes': axes, 'stds': stds})
    
    # visual
    draw_axes(mean, axes, stds, mesh_ls[idx], is_export=True, export_path=os.path.join('output', 'axes', f'fast_axes_{idx}.png'))
    print('-'*50)
    print(f'origin: {mean}\naxes: {axes}\nstds: {stds}')
    

--------------------------------------------------
origin: [ -19.35173802 -110.94776322   74.37263339]
axes: [[ 0.19903776 -0.14165672  0.96969962]
 [-0.92427362 -0.3560279   0.13770409]
 [-0.32573341  0.92367608  0.20179257]]
stds: [80.52793393 32.42319736 23.02691387]
--------------------------------------------------
origin: [ -22.42135458 -127.11639593   64.8034745 ]
axes: [[ 0.20051408  0.02323243  0.97941531]
 [-0.94901286 -0.24361279  0.2000685 ]
 [-0.24324617  0.96959428  0.02679992]]
stds: [83.00320047 33.24360171 25.43468768]
--------------------------------------------------
origin: [ -19.68341878 -132.24813197   59.90065043]
axes: [[ 0.17702965  0.07655359  0.98122375]
 [-0.95378109 -0.23262645  0.19022768]
 [-0.24282121  0.9695486  -0.03183359]]
stds: [81.78863716 34.32191984 23.11622719]
--------------------------------------------------
origin: [ -18.23912556 -128.04792848   59.43811265]
axes: [[ 0.2030801   0.13585729  0.96969133]
 [-0.94794561 -0.22078855  0.22945922]


In [83]:
axes_ls

[{'mean': array([ -19.35173802, -110.94776322,   74.37263339]),
  'axes': array([[ 0.19903776, -0.14165672,  0.96969962],
         [-0.92427362, -0.3560279 ,  0.13770409],
         [-0.32573341,  0.92367608,  0.20179257]]),
  'stds': array([80.52793393, 32.42319736, 23.02691387])},
 {'mean': array([ -22.42135458, -127.11639593,   64.8034745 ]),
  'axes': array([[ 0.20051408,  0.02323243,  0.97941531],
         [-0.94901286, -0.24361279,  0.2000685 ],
         [-0.24324617,  0.96959428,  0.02679992]]),
  'stds': array([83.00320047, 33.24360171, 25.43468768])},
 {'mean': array([ -19.68341878, -132.24813197,   59.90065043]),
  'axes': array([[ 0.17702965,  0.07655359,  0.98122375],
         [-0.95378109, -0.23262645,  0.19022768],
         [-0.24282121,  0.9695486 , -0.03183359]]),
  'stds': array([81.78863716, 34.32191984, 23.11622719])},
 {'mean': array([ -18.23912556, -128.04792848,   59.43811265]),
  'axes': array([[ 0.2030801 ,  0.13585729,  0.96969133],
         [-0.94794561, -0.220

In [86]:
from mesh4d import utils

utils.save_pkl_object(axes_ls, 'output/axes', 'fast_axes')

## Convert to local frames

$$M (a_i + t) = e(i)$$
$$\Rightarrow M (A + t) = I$$
$$\Rightarrow M = (A + t)^{-1}$$

In [7]:
def coord_cartesian2homo(vertices):
    shape = list(vertices.shape)
    shape[1] += 1

    vertices_homo = np.ones(shape)
    vertices_homo[:, :3] = vertices
    return vertices_homo

In [79]:
def coord_homo2cartesian(vertices_homo):
    return vertices_homo[:, :3] / vertices_homo[:, [-1]]

In [59]:
def transformation_matrix2global(axes, origin):
    """e(i) -> a_i + t"""
    matrix = np.eye(4)
    matrix[:3, :3] = axes.T
    matrix[:3, 3] = origin
    
    return matrix

In [60]:
def transformation_matrix2local(axes, origin):
    """a_i + t -> e(i)"""
    matrix2golbal = transformation_matrix2global(axes, origin)
    return np.linalg.inv(matrix2golbal)

In [64]:
def transform(matrix, vertices):
    vertices_homo = coord_cartesian2homo(vertices)
    vertices_transform = (matrix @ vertices_homo.T).T
    return coord_homo2cartesian(vertices_transform)

In [81]:
arr, names = landmarks.to_array()
arr_transform = []

for idx in range(len(mesh_clip_ls)):
    matrix = transformation_matrix2local(axes_ls[idx]['axes'], axes_ls[idx]['mean'])
    mesh_transform = mesh_clip_ls[idx].transform(matrix, inplace=False)

    vertices = arr[idx]
    vertices_transform = transform(matrix, arr[idx])
    arr_transform.append(vertices_transform)

    # visual
    scene = pv.Plotter()
    scene.add_mesh(mesh_transform, opacity=0.5, color='goldenrod')
    scene.add_mesh(mesh_clip_ls[idx], opacity=0.5)
    scene.add_points(vertices, color='black', point_size=10, render_points_as_spheres=True)
    scene.add_points(vertices_transform, color='red', point_size=10, render_points_as_spheres=True)
    scene.screenshot(os.path.join('output', 'transform', f'fast_transform_{idx}.png'))

arr_transform = np.array(arr_transform)

In [82]:
from mesh4d import kps

landmarks_transform = kps.MarkerSet()
landmarks_transform.load_from_array(arr_transform, start_time=0, fps=landmarks.fps, index=names)
landmarks_transform.interp_field()
utils.save_pkl_object(landmarks_transform, 'output/transform', 'fast_transform')